### 정형데이터 - 우리가 지금까지 다룬 데이터  숫자로 되어 있는 계산할수 있는 데이터
### 비정형데이터 - 이미지, 사운드, 텍스트
### 정형데이터를 가지고 가장뛰어난 성능을내는 머신러닝 알고리즘 : 앙상블..
### 랜덤 포레스트 : 앙상블의 대표주자
### tree : random data - 
    부트스트랩 샘플 : 데이터세트에서 한개씩 랜덤하게 선택해서 훈련데이터셋을 만드는데. 중복이 될수 있고
    훈련세트는 데이터세트와 같게 만든다.. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
url = 'https://raw.githubusercontent.com/leekyuyoung20221226/python/main/data/wine_data'
wine = pd.read_csv(url)
wine.head(1)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0


In [5]:
X = wine.iloc[:,:-1].to_numpy()
Y = wine.iloc[:,-1].to_numpy()

In [8]:
X.shape, Y.shape

((6497, 3), (6497,))

In [9]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [10]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1,random_state=0)

In [13]:
scores =  cross_validate(rf,x_train,y_train,return_train_score=True)

In [14]:
scores

{'fit_time': array([0.15558386, 0.14162064, 0.12965322, 0.12666178, 0.14461327]),
 'score_time': array([0.02592897, 0.02293825, 0.02194166, 0.0209434 , 0.0269289 ]),
 'test_score': array([0.88269231, 0.86826923, 0.88739172, 0.89027911, 0.88931665]),
 'train_score': array([0.99807554, 0.99831609, 0.9973545 , 0.9973545 , 0.9978355 ])}

In [15]:
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9977872242245589, 0.883589805286148)

In [18]:
# 특성 중요도
rf.fit(x_train,y_train)
rf.feature_importances_, wine.columns[:-1]

(array([0.23029487, 0.5028204 , 0.26688473]),
 Index(['alcohol', 'sugar', 'pH'], dtype='object'))

In [19]:
# 랜덤포레스트 VS 디시전 트리
# 랜덤하게 피처(특성,컬럼)를 선택하기때문에.. 한쪽에 쏠리는 현상을 방지한다... 좀더 많은 특성에 기회를 부여
# 자체적으로 모델을 평가하는 기능 -- 중복을 허용해서 부트스트랩 샘플을 만들면 남는 샘플이 있음  OOB(out of bag)
rf = RandomForestClassifier(n_jobs=-1,random_state=0, oob_score=True)
rf.fit(x_train,y_train)
rf.oob_score_

0.8926303636713488

In [20]:
# --> 교차검증을 대신 할수 있음

# 엑스트라 트리 
    100개의 트리를 훈련
    전체특징중에 일부를 랜덤하게 선택해서 노드를 분할하는데 사용
    부트스트랩 샘플을 사용하지 않는다 - 전체 데이터를 사용, 노드분할을 할때 무작위(최적의 노드를 찾는게 아니라)
    특성을 무작위로 분할... 성능은 낮아지지만.. 과적합을 예방할수 있음

In [21]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=0)
scores = cross_validate(et,x_train,y_train,return_train_score=True)
np.mean(scores['train_score']),np.mean(scores['test_score'])

(0.9978834474624707, 0.880317428000296)

In [22]:
# 랜덤포레스트와 결과가 비슷 why?  특성이 별로 없어서... 3
# 수행속도가 빠르다.
et.fit(x_train,y_train)
et.feature_importances_

array([0.20298962, 0.51938012, 0.27763026])

# 그레이던트 부스팅
    숲을 이루는 결정트리를 만들때.. 깊이가 얖은 트리를 사용
    기본이 깊이가 3인 트리 100개를 사용
    과적합에 강하다.
    경사하강법이 사용되었고..
    분류 : 로지스틱 손실 함수(시그모이드) -- 확률로 변경하기위해서.. 
    회귀 : 평균 제곱 오차

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
gbc =  GradientBoostingClassifier()
scores = cross_validate(gbc,x_train,y_train,return_train_score=True, n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.880123155562174, 0.8656909380321315)

In [26]:
gbc = GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=0)
scores = cross_validate(gbc,x_train,y_train,return_train_score=True, n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9385222836750382, 0.8708875027763382)

In [27]:
gbc.fit(x_train,y_train)
gbc.feature_importances_

array([0.1550395 , 0.68721774, 0.15774276])

### 지금까지 숲을 구성할때. 사용한 트리는(트리의 훈련데이터는) 원래 훈련 데이터 전부 다 사용
### subsample= 1.0  전체데이터를 다 사용
### 이 값을 1.0보다 작게 만들면  일부 데이터만 사용 - 일부 셈플만 랜덤하게 선택
### 확률적 경사하강법 or 미니배치 경사 하강법 과 비슷하게 된다.
### 랜덤포레스트보다 속도가 느리다.. n_job = -1 없음

# 히스토그램 기반 그레이디언트 부스팅
    그레이디언트 부스팅의 속도와 성능을 개선한 알고리즘
    입력특성을 256개의 구간으로 먼저 나눈다 - 노드를 분할할때 최적의 분할을 빠르게 찾을수 있음
    트리의 개수를 지정하는거 대신에 반복학습 max_iter 를 제공

In [49]:
from sklearn.ensemble import HistGradientBoostingClassifier
hbc =  HistGradientBoostingClassifier(random_state=0,max_iter=100)
scores = cross_validate(hbc, x_train,y_train,return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9284684595245096, 0.8737748944991486)

In [37]:
# 중요도
from sklearn.inspection import permutation_importance
hbc.fit(x_train,y_train)
result = permutation_importance(hbc, x_train,y_train,n_repeats=10,random_state=0,n_jobs=-1)
result['importances_mean']

array([0.09184145, 0.24083125, 0.08324033])

In [41]:
import sklearn
sklearn.__version__

'1.2.0'

In [44]:
# 사이킷런 업그레이드 - 필요시
!pip install --user -U scikit-learn

# XGBoost 그레이디언트 부스팅 알고리즘을 사용한 모델(사이킷런이 아님)
# lightgbm : 그레이디언트 부스팅 알고리즘을 사용한 모델 마이크로 소프트에서 만든 알고리즘(최신기술 반영)

In [51]:
# 가상머신... 그런데.. 코렙은 기본 탑재
!pip install xgboost

In [57]:
from xgboost import XGBClassifier
xgb =  XGBClassifier(random_state = 0)
scores = cross_validate(xgb,x_train,y_train,return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9517029349360356, 0.875314466572888)

In [58]:
!pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 21.6 MB/s eta 0:00:00


In [59]:
from lightgbm import LGBMClassifier
lgbmc =  LGBMClassifier(random_state=0)
scores = cross_validate(lgbmc,x_train,y_train,return_train_score=True)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9320281523553113, 0.876660250240616)

In [60]:
# https://xgboost.ai/
# https://github.com/microsoft/LightGBM